In [1]:
from typing import Any

import numpy
import polars
from datasets import Dataset, load_dataset
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
dataset = load_dataset(
	"csv", data_dir="../data", data_files={"train": "train.csv", "test": "test.csv"}, keep_in_memory=True, num_proc=8
)

In [3]:
index_columns: list = ["data_ID", "player_ID"]
label_columns: list = ["gender", "hold racket handed", "play years", "level"]
data_columns: list = list(set(dataset["train"].column_names).difference(set(index_columns + label_columns)))

In [4]:
data_columns

['gy_var',
 'gx_var',
 'a_kurt',
 'g_fft',
 'gx_mean',
 'a_psd',
 'g_entropy',
 'az_var',
 'g_min',
 'gz_var',
 'a_fft',
 'g_kurt',
 'gy_mean',
 'ax_mean',
 'az_mean',
 'a_max',
 'gx_rms',
 'a_mean',
 'a_min',
 'ax_rms',
 'gz_rms',
 'g_psd',
 'g_mean',
 'gz_mean',
 'g_max',
 'ay_rms',
 'gy_rms',
 'a_skewn',
 'a_entropy',
 'ax_var',
 'az_rms',
 'g_skewn',
 'ay_mean',
 'ay_var']

In [5]:
scaler = StandardScaler()

In [6]:
train_dataset: polars.DataFrame = dataset["train"].to_polars()
train_data: numpy.ndarray = scaler.fit_transform(train_dataset[data_columns].to_numpy())
train_data, validation_data, t_label, v_label = train_test_split(
	train_data, train_dataset[label_columns].to_numpy(), test_size=0.1, random_state=37
)
train_label: dict[str, numpy.ndarray] = {label: t_label[:, i] for i, label in enumerate(label_columns)}
validation_label: dict[str, numpy.ndarray] = {label: v_label[:, i] for i, label in enumerate(label_columns)}

In [7]:
hyperparameter_grid: dict[str, list[Any]] = {
	"hidden_layer_sizes": [(50,), (100,), (200,), (50, 50), (100, 50), (100, 100)],
	"activation": ["identity", "logistic", "tanh", "relu"],
	"solver": ["adam", "sgd", "lbfgs"],
	"learning_rate": ["constant", "adaptive", "invscaling"],
	"learning_rate_init": [0.001, 0.01, 0.1],
	"max_iter": [200, 300, 400, 500],
	"alpha": [0.0001, 0.001, 0.01],
	"batch_size": ["auto", 32, 64, 128],
}

In [8]:
best_classifiers: dict = {}
for label in label_columns:
	print(f"Tune classifier for {label}")
	classifier = MLPClassifier(random_state=37)

	tuner = RandomizedSearchCV(
		classifier, hyperparameter_grid, n_iter=50, cv=5, scoring="roc_auc_ovr", n_jobs=-1, verbose=10
	)

	tuner.fit(train_data, train_label[label])

	best_classifiers[label] = tuner.best_estimator_

Tune classifier for gender
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Tune classifier for hold racket handed
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Tune classifier for play years
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Tune classifier for level
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/Users/hermeschen/Repo/tabletenis-player-classify/.venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:659: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/hermeschen/Repo/tabletenis-player-classify/.venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:659: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/hermeschen/Repo/tabletenis-player-classify/.venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:659: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/hermeschen/Repo/tabletenis-player-classify/.venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:659: ConvergenceWarning: Stochastic Optimize

In [9]:
test_data: polars.DataFrame = scaler.transform(dataset["test"].select_columns(data_columns).to_polars().to_numpy())

In [10]:
predictions: dict = {"data_ID": dataset["test"]["data_ID"]}
for label in label_columns:
	predictions[label] = best_classifiers[label].predict(test_data).tolist()

In [11]:
test_result = Dataset.from_dict(predictions)

In [12]:
test_result = test_result.map(
	lambda samples: {
		"play years_0": [1 if sample == 0 else 0 for sample in samples],
		"play years_1": [1 if sample == 1 else 0 for sample in samples],
		"play years_2": [1 if sample == 2 else 0 for sample in samples],
	},
	input_columns=["play years"],
	remove_columns=["play years"],
	batched=True,
	num_proc=8,
)

Map (num_proc=8):   0%|          | 0/7488 [00:00<?, ? examples/s]

In [13]:
test_result = test_result.map(
	lambda samples: {
		"level_0": [1 if sample == 0 else 0 for sample in samples],
		"level_1": [1 if sample == 1 else 0 for sample in samples],
		"level_2": [1 if sample == 2 else 0 for sample in samples],
	},
	input_columns=["level"],
	remove_columns=["level"],
	batched=True,
	num_proc=8,
)

Map (num_proc=8):   0%|          | 0/7488 [00:00<?, ? examples/s]

In [14]:
test_result

Dataset({
    features: ['data_ID', 'gender', 'hold racket handed', 'play years_0', 'play years_1', 'play years_2', 'level_0', 'level_1', 'level_2'],
    num_rows: 7488
})

In [15]:
test_result.to_csv("../dist/result_mp.csv", num_proc=8)